# Natural Language Processing Lab

This work is on spanish and it was made as a practice project on "Instituto de Computación - Facultad de Ingeniería - UdelaR, Montevideo Uruguay" for the subject: "Introducción al Procesamiento del Lenguaje Natural" (Introduction to NLP).

# Task 1

This task's objective is to work with different PLN tools on a tweet corpus (a noisy dataset that needs to be preprocesed) using NLP tools like spacy https://spacy.io/api and regular expressions to obtain linguistic information, then, we will be feeding this corpus to different machine learning methods to create a sentiment analysis classifier. 



In [ ]:
!pip install spacy

# Parte 1 - Carga y preprocesamiento del corpus

Hay muchas formas de trabajar con conjuntos de datos (por ejemplo con [Pandas](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)), en esta tarea vamos a utilizar la biblioteca [csv](https://docs.python.org/3/library/csv.html). El resultado será una lista de n-uplas, cada una de las cuales correponde a una fila del .csv (incluso el cabezal, la primera línea).


In [ ]:
import csv
import random

# Load the train.csv file

with open('train.csv', newline='', encoding="utf-8") as corpus_csv:
  reader = csv.reader(corpus_csv)
  next(reader) # Saltea el cabezal del archivo
  train_set = [x for x in reader]

# Choose a random tweet and print it with its category

random_tweet = random.choice(train_set)
print(f"El tweet tiene id: {random_tweet[0]}")
print(f"El tweet es: {random_tweet[1]}")
print(f"y su categoría: {random_tweet[2]}")

Una vez cargado el corpus, que por tratarse de un cnjunto de tweets tiene características particulares, deben analizar varios ejemplos para definir qué preprocesamiento consideran necesario. Algunos ejemplos de limpieza de los datos que puede ser interesante aplciar:

* Eliminar url o remplazarlas por algo genérico, como el texto "(URL)" (*La juez envía a prisión al exdirector de Trabajo andaluz imputado por el caso de los ERE falsos http://t.co/OadFKKj2) -> La juez envía a prisión al exdirector de Trabajo andaluz imputado por el caso de los ERE falsos **(URL)** )*

* Remplazar menciones a usuarios por algo genérico, como "USUARIO" (*@earthlymama15 ahi voy , cambiando hábitos y haciendo un nuevo estilo de vida -> **USUARIO** ahi voy , cambiando hábitos y haciendo un nuevo estilo de vida*)

* Remplazar abreviaturas comunes por el término original (*A los okupas no les gusta el edificio de Concepcion Jeronima xq esta en obras. -> A los okupas no les gusta el edificio de Concepcion Jeronima **porque** esta en obras.*)

* Homogeneizar algunas expresiones frencuentes en este tipo de texto, como risas (*@LaMachaCR Ve que sos de las pocas positivas en mi TL, no me decepciones jajaja. -> @LaMachaCR Ve que sos de las pocas positivas en mi TL, no me decepciones **jaja**. // @ViKo_LH mira hijo de puta me cago en tu puta madre JAJAJAJAJAJAJJAAJAJA ya podré ir en 2018, ya...  -> @ViKo_LH mira hijo de puta me cago en tu puta madre **jaja** ya podré ir en 2018, ya...*) 

A continuación mostramos algunos ejemplos de uso de la biblioteca [re](https://docs.python.org/3/library/re.html) para trabajar con expresiones regulares (no es obligatorio usarla).

In [ ]:
# Example of substitutions using regular expressions.
import re

sentence = 'Si mañana vengo voy a ver si consigo que #mañana sea hoy'
sentence = re.sub('mañana', 'hoy', sentence)
print(sentence)
sentence = re.sub('#\w+', 'HASHTAG', sentence)
print(sentence)

In [5]:
import nltk
from nltk.tokenize import word_tokenize

## Parte 1

Para esta primera parte que consta de preprocesar el texto, ademas de con considerar los 4 ejemplos dados: 
  - Remplazar url's por "URL".
  - Remplazar menciones a usuarios por "USUARIO".
  - Remplazar abreviaturas.
  - Homogeneizar algunas expresiones frencuentes (la risa)

Tambien se consideraron:
   - Transformar todas las mayusculas en minusculas para asi homogenizar todas las palabras.
   - Remover todos los acentos.
   - Homogenizar insultos o groserias por "GROSERIA"
   - Reemplazar hashtags por "HASHTAG"
   - Ante repeticiones de una misma letra, 3 o mas veces seguidas, reemplazar las mismas por la letra en cuestion.
   - Reemplazar 2 o mas espacios seguidos por un unico espacio.

Se definio para la mayoria de estas opciones de prepocesamiento una funcion, las cuales seran llamadas en una funcion principal que las agrupa, llamada "procesar_tweet".


Otras consideraciones:
   - Para algunas expresiones regulares nos auxiliamos de ciertos patrones genericos disponibles en la web.
   - A pesar de que normalizaremos los tweets, dejandolos en minuscula, algunos patrones consideran letras mayusculas en caso de que se opte mas adelante no realizar el cambio a minusculas.

In [6]:
# Código de los estudiantes para resolver la parte 1
# Incluir comentarios que documenten el código

# reemplazar_URL(texto):
#Reemplaza las URL del texto recibido por parámetro por el string "URL".
#Retorna además un identificador para controlar si hubo algún cambio en el tweet
#analizado, útil para realizar recorridas de testeo 
def reemplazar_URL(texto):
    texto2 = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "URL", texto)
    cambio = (texto2!=texto)
    return [texto2,cambio]



# reemplazar_Usuario(texto):
#Reemplaza las menciones a usuarios del texto recibido por parámetro por el string "usuario".
#Retorna además un identificador para controlar si hubo algún cambio en el tweet
#analizado, útil para realizar recorridas de testeo 
def reemplazar_Usuario(texto): 
    regex2 = r"@(\w+)"
    texto2 = re.sub(regex2,"usuario", texto)
    cambio = (texto2!=texto)
    return [texto2,cambio]



# reemplazar_Abreviaturas(texto):
#Remplazar abreviaturas comunes por el término original.
#Retorna además un identificador para controlar si hubo algún cambio en el tweet
#analizado, útil para realizar recorridas de testeo 
def reemplazar_Abreviaturas(texto):
    texto2 = re.sub(r'[\s^][xX][qQ][\s$]',' porque ', texto)
    texto2 = re.sub(r'[\s^][pP](\s)*[qQ][\s$]',' porque ', texto2)
    texto2 = re.sub(r'porq',' porque ', texto2)
    texto2 = re.sub('[\s^][xX][\s$]',' por ', texto2)
    texto2 = re.sub('[\s^][qQ][\s$]',' que ', texto2)
    texto2 = re.sub('[\s^][kK][\s$]',' que ', texto2)
    texto2 = re.sub('[\s^][bB][nN][\s$]',' bien ', texto2)  
    texto2 = re.sub('[\s^][tT][mM][bB][\s$]',' tambien ', texto2)  

    # Consideramos que no corresponde reemplazar RT
    # como una abreviatura ya que puede cambiar el significado semántico de la oración,
    # y lo borramos directamente.
    texto2 = re.sub('[\s^][rR][tT][\s$]',' ', texto2)  

    texto2 = re.sub('[\s^][bB][bB][\?*\s$]',' bebé ', texto2)  
    texto2 = re.sub('[\s^][bB][bB][sS][\?*\s$]',' bebés ', texto2)  
    texto2 = re.sub('[\s^][vV][sS][\s$]',' versus ', texto2)  
    texto2 = re.sub('[\s^][cC][\s$]',' se ', texto2)  
    texto2 = re.sub('[\s^]\+[\s$]',' mas ', texto2)  
    texto2 = re.sub('[\s^][dD][\s$]',' de ', texto2)  
    texto2 = re.sub('[\s^][dD][lL][\s$]',' del ', texto2)  
    texto2 = re.sub('[\s^][tT][aA][\s$]',' está ', texto2)  
    texto2 = re.sub('[\s^][pP][aA][\s$]',' para ', texto2)  
    texto2 = re.sub('[\s^][pP][sS][\?*\.*\,*\s$]',' pues ', texto2)  
    texto2 = re.sub('[\s^][mM][\s$]',' me ', texto2)  
    texto2 = re.sub('[\s^][cC][sS][mM][\s$]',' conchesumare ', texto2) 
    texto2 = re.sub('[\s^][gG]ral[\s.$]',' general ', texto2) 
    texto2 = re.sub('[\s^][dD][rR][.\s$]',' doctor ', texto2) 
    cambio = (texto2!=texto)
    return [texto2,cambio]



# reemplazar_Risa(texto):
#Remplazar en este caso las variantes posibles de una risa (expresion frecuente) por el String "jaja".
#Retorna además un identificador para controlar si hubo algún cambio en el tweet
#analizado, útil para realizar recorridas de testeo 
def reemplazar_Risa(texto):
    er_risa = r'\b([aA]+[jJ]+[aA]+[aAjJ]*|[jJ]+[jaJA]+[jJ]+[jaJA]*|[aA]+[hH]+[aA]+[aAhH]*|[hH]+[haHA]+[hH]+[haHA]*|[oO]?[lL]+[oO]+[lL]+[oLOl]*|[aA]*[jaJA]+[jJ][jJAa]*|[eE]+[jJ]+[eE]+[eEjJ]*|[jJ]+[jeJE]+[jJ]+[jeJE]*|[eE]+[hH]+[eE]+[eEhH]*|[hH]+[heHE]+[hH]+[heHE]*|[eE]*[jeJE]+[jJ][jJeE]*)\b'
    texto2 = re.sub(er_risa,' jaja ', texto)
    cambio = (texto2!=texto)
    return [texto2,cambio]



# strip_accents(texto)
#Remueve los acentos del texto
def strip_accents(texto):
    texto2 = re.sub("á","a",texto)
    texto2 = re.sub("é","e",texto2)
    texto2 = re.sub("í","i",texto2)
    texto2 = re.sub("ó","o",texto2)
    texto2 = re.sub("ú","u",texto2) 
    return texto2



# reemplazar_Hashtags
# reemplazamos hashtags del texto por el string "HASHTAG".
# Retorna además un identificador para controlar si hubo algún cambio en el tweet
# analizado, útil para realizar recorridas de testeo 
def reemplazar_Hashtags(texto):
    er_hashtags = r'#\S+'
    texto2 = re.sub(er_hashtags,'HASHTAG', texto)
    cambio = (texto2!=texto)
    return [texto2,cambio]


# reemplazar_repeticiones(texto)
# Reemplaza las repeticiones de 3 o más veces la misma letra por una sola en "texto"
def reemplazar_repeticiones(texto):
    texto = re.sub("aaa[a]*","a",texto)
    texto = re.sub("bbb[b]*","b",texto)
    texto = re.sub("ccc[c]*","c",texto)
    texto = re.sub("ddd[a]*","d",texto)
    texto = re.sub("eee[e]*","e",texto)
    texto = re.sub("fff[f]*","f",texto)
    texto = re.sub("ggg[g]*","g",texto)
    texto = re.sub("hhh[h]*","h",texto)
    texto = re.sub("iii[i]*","i",texto)
    texto = re.sub("jjj[j]*","j",texto)
    texto = re.sub("kkk[k]*","k",texto)
    texto = re.sub("lll[l]*","l",texto)
    texto = re.sub("mmm[m]*","m",texto)
    texto = re.sub("nnn[n]*","n",texto)
    texto = re.sub("ooo[o]*","o",texto)
    texto = re.sub("ppp[p]*","p",texto)
    texto = re.sub("qqq[q]*","q",texto)
    texto = re.sub("rrr[r]*","r",texto)
    texto = re.sub("sss[s]*","s",texto)
    texto = re.sub("ttt[t]*","t",texto)
    texto = re.sub("uuu[u]*","u",texto)
    texto = re.sub("vvv[v]*","v",texto)
    texto = re.sub("www[w]*","w",texto)  #Luego de Urls
    texto = re.sub("xxx[x]*","x",texto)
    texto = re.sub("yyy[y]*","y",texto)
    texto = re.sub("zzz[z]*","z",texto)
    return texto


# reemplazar_Groserias(texto)
# Reemplaza las groserías de "texto" por el string "GROSERIA"
# Retorna además un identificador para controlar si hubo algún cambio en el tweet
# analizado, útil para realizar recorridas de testeo 
def reemplazar_Groserias(texto):
    regex1 = r'hij[oa]+[de]*[p]+[u]+[t]+[a]+|gilipollas|gilipolleces|bolud[oa]|mierda|mariconadas|estupid[oa]|estupide(z|ces)'
    regex2 = r'\sput[oa]\s|marica\s'
    texto2 = re.sub(regex1,' GROSERIA ', texto)
    texto2 = re.sub(regex2,' GROSERIA ', texto2)
    cambio = (texto2!=texto)
    return [texto2,cambio]


#Removemos 2 o mas espacios seguidos
def remover_espacios(texto):
  patron_espacios = r'\s+'
  texto = re.sub(patron_espacios, " ", texto)
  return texto

# remover_puntuacion(texto): Se reemplazan repeticiones redundantes de signos de puntuacion
# Ejemplos: ??? -> ?, !! -> !
# No quitamos comillas, signos de interrogación, exclamación, asteriscos, etc
# porque consideramos que pueden llegar a ser indicadores de sarcasmo, pudiendo
# llegar a ser tenidos en cuenta a la hora de realizar el análisis semántico, 
# en el siguiente laboratorio. 

def remover_puntuacion(texto):
    signos_puntuacion = r'[;\{\}/\[\]()\\]+'
    exclamaciones = r'\!+'
    interrogaciones = r'\?+'
    puntos = r'\.\.+'
    dos_puntos = r'\:+'
    comas = r'\,+'
    texto2 = re.sub(signos_puntuacion,'', texto)
    texto2 = re.sub(exclamaciones,'!', texto2)   #No se eliminan "!" ni "?" ya que son utiles para que el parser reconozca correctamente
                                                 #interjecciones 
    texto2 = re.sub(interrogaciones,'?', texto2)
    texto2 = re.sub(comas,',', texto2)
    texto2 = re.sub(dos_puntos,' ', texto2)           #Se eliminan los dos puntos, por problemas comentados en la parte 3
    texto2 = re.sub(puntos,'.', texto2)
    return texto2

#procesar_tweet(tweet):
# Aplicamos todas las funciones anteriores para procesar un tweet 
# y retornamos el resultado.
def procesar_tweet(tweet):
    contenido = ""
    contenido = tweet
    contenido = contenido.lower()
    contenido = strip_accents(contenido)
    cambio = True
    contenido,cambio = reemplazar_Hashtags(contenido)
    contenido,cambio = reemplazar_Usuario(contenido)
    contenido,cambio = reemplazar_URL(contenido)
    contenido,cambio = reemplazar_Groserias(contenido)
    contenido,cambio = reemplazar_Abreviaturas(contenido)
    contenido = reemplazar_repeticiones(contenido)
    contenido = remover_puntuacion(contenido)
    contenido,cambio = reemplazar_Risa(contenido)
    contenido = remover_espacios(contenido)
    return contenido

# Parte 2 - Herramientas de análisis lingüístico

Vamos a usar la biblioteca [Spacy](https://spacy.io/) (en particular, funciones para [análisis lingüístico](https://spacy.io/usage/linguistic-features)) para obtener informaicón lingüística de las palabras y oraciones de un texto.
Existen diferentes modelos disponibles para trabajar con textos en [español](https://spacy.io/models/es), incluimos un ejemplo de uso del modelo es_dep_news_trf.

En esta parte deben desplegar para cada palabra de un texto:

* lema
* POS-tag

y para cada oración:

* raíz
* sintagmas nominales y preposicionales mayores (constituyentes de la oración)
* función sintáctica de cada constituyente identificado


In [ ]:
# Carga del modelo para español es_dep_news_trf
!pip install https://huggingface.co/spacy/es_dep_news_trf/resolve/main/es_dep_news_trf-any-py3-none-any.whl
!pip install spacy-transformers

In [8]:
import spacy
from spacy import displacy
import spacy_transformers
analisisLing = spacy.load("es_dep_news_trf")

In [ ]:
 # Ejemplo de análisis lingüístico con spacy

ejemplo1 = analisisLing("El director del liceo fue sumariado con separación del cargo por “insubordinación”, lo que provocó que el conflicto se agudizara.")
    
for token in ejemplo1:
    print(token.text, token.pos_, token.dep_, token.head)
    
displacy.render(ejemplo1, style="dep", jupyter=True)    

In [10]:
# Código de los estudiantes para resolver la parte 2
# Incluir comentarios que documenten el código

# mostrarAnalisis(texto):   #Funcion brindada como ejemplo en la letra.
# Se analiza "texto" utilizando spacy y se imprimen en orden:
# palabra, POS-TAG, tipo de dependencia y nodo padre en el árbol sintáctico
def mostrarAnalisis(texto):
  ejemplo1 = analisisLing(texto)
  for token in ejemplo1:
      print(token.text, token.pos_, token.dep_, token.head)


######################## Analisis de Palabra ###################################

# pos_tag(texto):
# Devuelve una lista con las pos-tags de cada palabra de "texto"
def pos_tag(texto):
    analisis = analisisLing(texto)
    ret = []
    for token in analisis:
      ret.append(token.pos_)
    return ret


# lemas(texto):
# Devuelve una lista con todos los lemmas de cada palabra de "texto"
def lemas(texto):
    analisis = analisisLing(texto)
    ret = []
    for token in analisis:
      ret.append(token.lemma_)
    return ret


#Devuelve la lista de 3-uplas (palabra, POS-tag, lema). Para analizar salidas en Parte 3.
def lema_y_pos_tag(texto):
  aux = re.findall(r"[\w']+|[.,:!?;]", texto)   #Para que retorne tambien los signos de puntuacion. Split() no lo hace.
  tags = pos_tag(texto)
  lem = lemas(texto)
  x = zip(aux,tags,lem)
  return x
######################## Analisis de Oracion ###################################

# raiz(texto):
# Retorna las raíces obtenidas del análisis sintáctico de "texto".
def raiz(texto):
  analisis = analisisLing(texto)
  ret = ""
  rec = []
  for token in analisis:
    ret = token.dep_
    if (ret == 'ROOT'):
      rec.append(token.text)
  return rec


#Retorna una lista con los sintagmas nominales y preposicionales mayores de la oración, es decir, aquellos sintagmas
#que cumplen funciones sintácticas (reconocida por el analizador) respecto al verbo raíz (o principal).
#Recorremos los verbos de la oración y seleccionamos los sintagmas en su conjunto 
#"children", que indica los hijos directos en el arbol

def SintagmasMayores(doc):
  ret = []
  analisis = analisisLing(doc)
  aux = doc.split()
  raices = []
  for token in analisis:
    if token.dep_ == "ROOT" and token.pos_ == "VERB": # Como spacy generaliza el análisis entre varios idiomas, no solo fija la raíz en el verbo principal, ya que hay casos donde no lo es.
      raices.append(token)                            # Por lo tanto, debemos controlar que además de ser la raíz también es el verbo principal de la oración.
                                                      # Si el verbo principal no está marcado como raíz, no tenemos forma de identificarlo como núcleo verbal de la oración. Asumimos que el
                                                      # Analizador sintáctico identifica al verbo principal como "ROOT".

  for raiz in raices:  #Tomamos ese elemento, deberíamos tener uno por cada oración independiente, donde spacy separa oraciones a través del punto, entre otras delimitaciones que realiza.
    for token in raiz.children:  #Analizamos todos los sintagmas mayores mediante una recorrida de sus dependencias.
        frase = ''.join([t.text_with_ws for t in token.subtree])
        prep = False
        agregar = ''
        for elem in token.subtree:
          if (elem.pos_ == "ADP"):
            prep = True
          break
        if (prep):
          agregar = 'SP: '
        else:
          agregar = 'SN: '
        if not (sintagmaEnConjunto(frase,ret)): #Para no generar sintagmas repetidos
          if token.dep_ == "nsubj":        
            ret.append([frase,agregar+"Sujeto " + token.dep_])  #Sintagmas nominales mayores con función sintáctica sujeto
          elif token.dep_ == "obl":
            ret.append([frase,agregar+"Adjunto " + token.dep_])   #Sintagmas mayores con función sintáctica objeto indirecto, reconocidos por spacy     
          elif token.dep_ == "iobj":
            ret.append([frase,agregar+"Objeto Indirecto "+ token.dep_])                    
          elif token.dep_ in ["obj"]:        #En caso de tener la etiqueta "obj", tenemos multiples variantes
            for token2 in token.subtree: 
              if token2.dep_ == "case":                       #Si hay una preposición al inicio del subárbol
                if (token2.text in ["de","con","por"]):
                   ret.append([frase,agregar+"Complemento de Régimen " + token.dep_])  #En caso de que la preposición sea "de", "con", "a" o "por", se trata de un Complemento de régimen.
                elif (token2.text == 'a'): #La preposición "a" se utiliza tanto en complemento de régimen como en objeto indirecto
                  es_complemento = False
                  for token3 in raiz.children:
                    if(token3.pos_ == "PRON") and (token3.dep_ == "expl:pv"): #Si se utiliza un pronombre, entonces es un Objeto Indirecto
                      es_complemento = True
                      break
                    elif (token3.pos_ == "VERB"):
                      es_complemento = False
                      break
                  if es_complemento:                      
                    ret.append([frase,agregar+"Complemento de Régimen " + token.dep_])
                  else:
                    ret.append([frase,agregar+"Objeto Indirecto "+ token.dep_])
                else:
                  ret.append([frase,agregar+"Objeto Indirecto " + token.dep_])  #En otro caso, es (o es parte) de un objeto indirecto.
                break
              else:
                frase2 = ''.join([str(elem) for elem in frase.split()])
                if frase2 in ["Le","le","La","la","Lo","lo","Los","los","Las","las","Les","les"]:  #Sino, será un pronombre (objeto indirecto) o un objeto directo
                  ret.append([frase,agregar+"Objeto Indirecto " + token.dep_])
                else:
                  ret.append([frase,agregar+"Objeto directo " + token.dep_])   
                break
          elif token.dep_ == "expl:pv":  #Etiqueta utilizada por el parser para identificar pronombres reflexivos.
            ret.append([frase,agregar+"Pronombre reflexivo " + token.dep_])

  for sint in ret:  #Nos quedamos con los mayores
    sintagma,dep = sint
    for sint2 in ret:
      sintagma2,dep2 = sint2
      if sintagma2 != sintagma:
        if sintagma2 in sintagma:
          ret.remove(sint2)
          break
  


  return ret


def sintagmaEnConjunto(sintagma,conjunto):
  ret = False
  for elemento in conjunto:
    sint,dep = elemento
    if(sintagma == sint):
      ret = True
      break
  return ret

In [11]:
#Probamos la función de la parte anterior para obtener sintagmas nominales y preposicionales mayores 
#Junto con la función sintáctica de cada componente identificado.
#Presentamos cinco casos vistos en el teórico (Diapositivas de clase: Introducción a la gramática del español)
orac1 = "Un árbol frondoso nos protege del sol."     #Sujeto
orac2 = "Vimos un árbol frondoso."                   #Objeto Directo
orac3 = "Le cortaron una rama a un árbol frondoso."  #Objeto Indirecto
orac4 = "Soñé con un árbol frondoso."                #Complemento de Regimen
orac5 = "Nos sentamos bajo un árbol frondoso."       #Adjunto  
orac6 = "El hijo de Luis vive en Madrid"             #Identificamos un adjunto que es prototípicamente un SP.
orac7 = "No se acostumbra a su nueva vida."          #Caso de complemento de régimen con preposición "a".
orac8 = "Es muy propenso a las infecciones respiratorias en primavera" #Sintagma Adjetival donde el núcleo de la oración es "propenso", por lo que el
                                                                       #analizador sintáctico no reconoce al verbo como raíz y no es posible identificar sus constituyentes, por lo tanto
                                                                       #nuestra función no retorna "en primavera" como sintagma preposicional mayor.

print("Ejemplo 1:")
rooot = raiz(orac1)
print("Oracion1: "+ orac1)
print("Lemas y POS-TAGS:" + str(list(lema_y_pos_tag(orac1))))
print("raices:" +str(rooot) +"\nSintagmas mayores oracion 1: " +  str(SintagmasMayores(orac1)))


print("\n\nEjemplo 2:")
rooot = raiz(orac2)
print("Oracion 2: "+ orac2)
print("Lemas y POS-TAGS:" + str(list(lema_y_pos_tag(orac2))))
print( "raices:" +str(rooot) +"\nSintagmas mayores oracion 2: " +  str(SintagmasMayores(orac2)))


print("\n\nEjemplo 3:")
rooot = raiz(orac3)
print("Oracion 3: "+ orac3)
print("Lemas y POS-TAGS:" + str(list(lema_y_pos_tag(orac3))))
print( "raices:" +str(rooot) +"\nSintagmas mayores oracion 3: " +  str(SintagmasMayores(orac3)))

print("\n\nEjemplo 4:")
rooot = raiz(orac4)
print("Oracion 4: "+ orac4)
print("Lemas y POS-TAGS:" + str(list(lema_y_pos_tag(orac4))))
print( "raices:" +str(rooot) +"\nSintagmas mayores oracion 4: " +  str(SintagmasMayores(orac4)))

print("\n\nEjemplo 5:")
rooot = raiz(orac5)
print("Oracion 5: "+ orac5)
print("Lemas y POS-TAGS:" + str(list(lema_y_pos_tag(orac5))))
print("raices:" +str(rooot) +"\nSintagmas mayores oracion 5: " +  str(SintagmasMayores(orac5)))

print("\n\nEjemplo 6:")
rooot = raiz(orac6)
print("Oracion 6: "+ orac6)
print("Lemas y POS-TAGS:" + str(list(lema_y_pos_tag(orac6))))
print("raices:" +str(rooot) +"\nSintagmas mayores oracion 6: " +  str(SintagmasMayores(orac6)))

print("\n\nEjemplo 7:")
rooot = raiz(orac7)
print("Oracion 7: "+ orac7)
print("Lemas y POS-TAGS:" + str(list(lema_y_pos_tag(orac7))))
print("raices:" +str(rooot) +"\nSintagmas mayores oracion 7: " +  str(SintagmasMayores(orac7)))


print("\n\nEjemplo 8:")
rooot = raiz(orac8)
print("Oracion 8: "+ orac8)
print("Lemas y POS-TAGS:" + str(list(lema_y_pos_tag(orac8))))
print("raices:" +str(rooot) +"\nSintagmas mayores oracion 8: " +  str(SintagmasMayores(orac8)))

Ejemplo 1:
Oracion1: Un árbol frondoso nos protege del sol.
Lemas y POS-TAGS:[('Un', 'DET', 'uno'), ('árbol', 'NOUN', 'árbol'), ('frondoso', 'ADJ', 'frondoso'), ('nos', 'PRON', 'yo'), ('protege', 'VERB', 'proteger'), ('del', 'ADP', 'del'), ('sol', 'NOUN', 'sol'), ('.', 'PUNCT', '.')]
raices:['protege']
Sintagmas mayores oracion 1: [['Un árbol frondoso ', 'SN: Sujeto nsubj'], ['nos ', 'SN: Objeto directo obj'], ['del sol', 'SP: Objeto Indirecto obj']]


Ejemplo 2:
Oracion 2: Vimos un árbol frondoso.
Lemas y POS-TAGS:[('Vimos', 'VERB', 'ver'), ('un', 'DET', 'uno'), ('árbol', 'NOUN', 'árbol'), ('frondoso', 'ADJ', 'frondoso'), ('.', 'PUNCT', '.')]
raices:['Vimos']
Sintagmas mayores oracion 2: [['un árbol frondoso', 'SN: Objeto directo obj']]


Ejemplo 3:
Oracion 3: Le cortaron una rama a un árbol frondoso.
Lemas y POS-TAGS:[('Le', 'PRON', 'él'), ('cortaron', 'VERB', 'cortar'), ('una', 'DET', 'uno'), ('rama', 'NOUN', 'rama'), ('a', 'ADP', 'a'), ('un', 'DET', 'uno'), ('árbol', 'NOUN', 'árbol

Observamos que al utilizar las etiquetas "obj" e "iobj" de las dependencias proporcionadas por el análisis de spacy para los objetos directo e indirecto, nuestra función no detecta los sintagmas mayores correctamente. Por lo tanto decidimos implementar un análisis posterior al de spacy para tratar de diferenciar correctamente los casos. Luego, implementamos una serie de casos de prueba obtenidos del teórico de introducción a la gramática del español, como se puede observar, en estos casos de prueba obtenemos correctamente cada rol indicado en el teórico.

Respecto a las oraciones subordinadas, consideramos que no es necesario identificarlas, puesto que solo se pide identificar los componentes que nuestro parser nos permite reconocer como sintagmas nominales o preposicionales mayores constituyentes de la oración.

Observamos, en el siguiente ejemplo, que la oración subordinada está marcada con una etiqueta "ccomp" que significa "clausal complement", es decir, un complemento de cláusula que funciona como objeto del verbo principal o como adjetivo, por lo que no se nos brinda la información suficiente para identificarlo como un sintagma Nominal o Preposicional.

In [12]:
orac8 = "Me dijo que no vendría."
print("\n\nEjemplo 8:")
rooot = raiz(orac8)
print("Oracion 8: "+ orac8)
print("Lemas y POS-TAGS:" + str(list(lema_y_pos_tag(orac8))))
print("raices:" +str(rooot) +"\nSintagmas mayores oracion 8: " +  str(SintagmasMayores(orac8)))



Ejemplo 8:
Oracion 8: Me dijo que no vendría.
Lemas y POS-TAGS:[('Me', 'PRON', 'yo'), ('dijo', 'VERB', 'decir'), ('que', 'SCONJ', 'que'), ('no', 'ADV', 'no'), ('vendría', 'VERB', 'venir'), ('.', 'PUNCT', '.')]
raices:['dijo']
Sintagmas mayores oracion 8: [['Me ', 'SN: Objeto Indirecto iobj']]


# Parte 3 - Análisis de tweets

Analice los siguiente tweets y comente qué problemas encuentra en las salidas del procesamiento lingüístico de la parte 2, habiendo previamente preprocesado los textos en base a la parte 1.

* *Un gran acierto d @jorgefdezpp el nombramiento d @Ignacos como nuevo Director Gral. de la Policía: un puesto central en democracia*

* *Les personas no somos eternas Lo más importante es el legado que dejamos, la enseñanza para toda una familia y una generación^*

* *@FanClubMASes: Éstas Navidades regala solidaridad:#positivegeneration (cont) http://t.co/BksU1DZv*

* *@DavidSummersHG: @Edurnity Oyeee...! A Madrid si quiero ir eh? Un besazo guapa! Vamossss ya contaba contigo si o si!!! Un besote enorme!!*




**TWEET 1:**

In [13]:
#Análisis con el Tweet Original:    #Funcion Nueva

tweet = "Un gran acierto d @jorgefdezpp el nombramiento d @Ignacos como nuevo Director Gral. de la Policía: un puesto central en democracia"
tweet_procesado = procesar_tweet(tweet)
pos = pos_tag(tweet_procesado)
cambio = False
sintagmas = SintagmasMayores(tweet_procesado)
root = raiz(tweet_procesado)
x = lema_y_pos_tag(tweet_procesado)
print("Tweet: " + str(tweet) + "\nTweet Procesado: " + str(tweet_procesado)  + "\n(palabra,POS-tag,lema): " + str(list(x))  + "\nRaiz: " + str(root) + "\n(Sintagmas Mayores,Funcion sintactica): " + str(sintagmas) )
analisisParte1 = analisisLing(tweet_procesado)
displacy.render(analisisParte1, style="dep", jupyter=True)  

Tweet: Un gran acierto d @jorgefdezpp el nombramiento d @Ignacos como nuevo Director Gral. de la Policía: un puesto central en democracia
Tweet Procesado: un gran acierto de usuario el nombramiento de usuario como nuevo director general de la policia un puesto central en democracia
(palabra,POS-tag,lema): [('un', 'DET', 'uno'), ('gran', 'ADJ', 'gran'), ('acierto', 'NOUN', 'acierto'), ('de', 'ADP', 'de'), ('usuario', 'NOUN', 'usuario'), ('el', 'DET', 'el'), ('nombramiento', 'NOUN', 'nombramiento'), ('de', 'ADP', 'de'), ('usuario', 'NOUN', 'usuario'), ('como', 'SCONJ', 'como'), ('nuevo', 'ADJ', 'nuevo'), ('director', 'NOUN', 'director'), ('general', 'ADJ', 'general'), ('de', 'ADP', 'de'), ('la', 'DET', 'el'), ('policia', 'PROPN', 'policia'), ('un', 'DET', 'uno'), ('puesto', 'NOUN', 'puesto'), ('central', 'ADJ', 'central'), ('en', 'ADP', 'en'), ('democracia', 'NOUN', 'democracia')]
Raiz: ['acierto']
(Sintagmas Mayores,Funcion sintactica): []


**Explicación tweet 1:**  \\

Al analizar el primer tweet, observamos que no tiene verbo, por lo tanto, nuestra funcion "SintagmasMayores(texto)" retornara una lista vacia, puesto que no habra sintagmas nominales y preposicionales que cumplan una funcion sintactica respecto a un verbo. Por otro lado, vemos que el tweet está compuesto por los consituyentes: "Un gran acierto d @jorgefdezpp", "el nombramiento d @Ignacos como nuevo Director Gral. de la Policía:" y  "un puesto central en democracia". Esto provoca que el analizador sintáctico interprete que el constituyente "un puesto central en democracia" está relacionado con "el nombramiento...", ya que "nombramiento" posee la dependencia de aposicional con "puesto", es decir, está describiendo dicho sustantivo, cuando en realidad no es así, puesto que se entiende que el puesto central en democracia es en realidad el de Director Gral. de la Policía.  

En cuanto al primer inconveniente de no tener verbo el tweet, la función para reconocer sintagmas nominales de la parte 2 no es útil, ya que el analizador sintáctico reconoce "acierto" como raíz de la oración, lo cual no es un verbo como fue mencionado.


**TWEET 2:**

In [14]:
tweet = "Les personas no somos eternas Lo más importante es el legado que dejamos, la enseñanza para toda una familia y una generación^"
tweet_procesado = procesar_tweet(tweet)
sintagmas = SintagmasMayores(tweet_procesado)
root = raiz(tweet_procesado)
x = lema_y_pos_tag(tweet_procesado)
print("Tweet: " + str(tweet) + "\nTweet Procesado: " + str(tweet_procesado)  + "\n(palabra,POS-tag,lema): " + str(list(x))  + "\nRaiz: " + str(root) + "\n(Sintagmas Mayores,Funcion sintactica): " + str(sintagmas) )
analisisParte2 = analisisLing(tweet_procesado)
displacy.render(analisisParte2, style="dep", jupyter=True)  

#Analizamos un texto similar al tweet nro 2, pero sin quitar las mayusculas para mostrar como el parser reconoce dos oraciones en lugar de una.
tweet = "Les personas no somos eternas Lo más importante es el legado que dejamos, la enseñanza para toda una familia y una generación^"
analisisParte2 = analisisLing(tweet)
displacy.render(analisisParte2, style="dep", jupyter=True)  

Tweet: Les personas no somos eternas Lo más importante es el legado que dejamos, la enseñanza para toda una familia y una generación^
Tweet Procesado: les personas no somos eternas lo mas importante es el legado que dejamos, la enseñanza para toda una familia y una generacion^
(palabra,POS-tag,lema): [('les', 'PRON', 'él'), ('personas', 'NOUN', 'persona'), ('no', 'ADV', 'no'), ('somos', 'AUX', 'ser'), ('eternas', 'ADJ', 'eterno'), ('lo', 'PRON', 'él'), ('mas', 'ADV', 'mas'), ('importante', 'ADJ', 'importante'), ('es', 'AUX', 'ser'), ('el', 'DET', 'el'), ('legado', 'NOUN', 'legado'), ('que', 'PRON', 'que'), ('dejamos', 'VERB', 'dejar'), (',', 'PUNCT', ','), ('la', 'DET', 'el'), ('enseñanza', 'NOUN', 'enseñanza'), ('para', 'ADP', 'para'), ('toda', 'DET', 'todo'), ('una', 'DET', 'uno'), ('familia', 'NOUN', 'familia'), ('y', 'CCONJ', 'y'), ('una', 'DET', 'uno'), ('generacion', 'NOUN', 'generacion^')]
Raiz: ['eternas']
(Sintagmas Mayores,Funcion sintactica): []


**Explicacion tweet 2:** \\
El pasaje a minúsculas de nuestro preprocesamiento impide al analizador sintáctico detectar que este tweet está compuesto de dos oraciones simples.
Nuevamente, al igual que con el tweet anterior, no se hallaron sintagmas nominales ni preposicionales que cumplan funciones sintácticas respecto al verbo principal debido a que la raíz es "eternas", es decir un adjetivo.

Otra observación que consideramos pertinente es el hecho de que al considerar dicho texto como una única oración, la raíz "eternas" se vincula con legado a través de la dependencia sintáctica "advcl" es decir cláusula adverbial, ya que modifica a dicho adjetivo. Sin embargo como mencionamos debería interpretarse como dos oraciones separadas. Como podemos observar, en el segundo análisis proporcionado no quitamos las mayúsculas, provocando que el analizador sintáctico separe en dos oraciones, lo cual es, desde nuestro punto de vista más adecuado. En dicho caso observamos también que desaparece la dependencia de conjunción entre "legado" y "generación", marcándose correctamente que la conjunción es entre generación y familia, debido a que la enseñanza aplica a ellos.

En cuanto a la afirmacion acerca de que el analizador sintactico detectar diferentes oraciones a traves de la presencia de ciertas palabras (o combinaciones de palabras) en Mayuscula, presentamos el siguiente codigo para reflejar dicho comportamiento. Cabe aclarar que se puede configurar distintos delimitantes de oraciones, pero nosotros utilizamos el que realiza Spacy por defecto. A su vez para el segundo ejemplo, vemos como la presencia de un "!" no implica necesariamente el final de una oracion, pues en el analisis se tienen en cuenta mas factores.

In [15]:
tweet = "Les personas no somos eternas Lo más importante es el legado que dejamos, la enseñanza para toda una familia y una generación^"
nlp = analisisLing(tweet)
oraciones = list(nlp.sents)   #retorna en la lista oraciones, las presentes en nlp, donde se analizo el tweet.

for oracion in oraciones:
    print(oracion)

print("\n\nVemos que sucede con otro String ante distintos signos de puntuacion: ")
tweet2 = "usuario: usuario oye.! a madrid si quiero ir eh? un besazo guapa! vamos ya contaba contigo si o si! un besote enorme!"
nlp2 = analisisLing(tweet2)
oraciones2 = list(nlp2.sents)

for oracion in oraciones2:
    # Print each sentence in the nlp with one sentence a line
    print(oracion)

Les personas no somos eternas
Lo más importante es el legado que dejamos, la enseñanza para toda una familia y una generación^


Vemos que sucede con otro String ante distintos signos de puntuacion: 
usuario: usuario oye.
! a madrid si quiero ir eh?
un besazo guapa!
vamos ya contaba contigo si o si! un besote enorme!


**TWEET 3**

In [16]:
tweet = "@FanClubMASes: Éstas Navidades regala solidaridad:#positivegeneration (cont) http://t.co/BksU1DZv"
tweet_procesado = procesar_tweet(tweet)
tweet_analizado = SintagmasMayores(tweet_procesado)
root = raiz(tweet_procesado)
x = lema_y_pos_tag(tweet_procesado)
print("Tweet: " + str(tweet) + "\nTweet Procesado: " + str(tweet_procesado)  + "\n(palabra,POS-tag,lema): " + str(list(x))  + "\nRaiz: " + str(root) + "\n(Sintagmas Mayores,Funcion sintactica): " + str(tweet_analizado) )
analisisParte3 = analisisLing(tweet_procesado)
displacy.render(analisisParte3, style="dep", jupyter=True) 

Tweet: @FanClubMASes: Éstas Navidades regala solidaridad:#positivegeneration (cont) http://t.co/BksU1DZv
Tweet Procesado: usuario estas navidades regala solidaridad HASHTAG cont URL
(palabra,POS-tag,lema): [('usuario', 'NOUN', 'usuario'), ('estas', 'DET', 'este'), ('navidades', 'NOUN', 'navidad'), ('regala', 'VERB', 'regalar'), ('solidaridad', 'NOUN', 'solidaridad'), ('HASHTAG', 'PROPN', 'HASHTAG'), ('cont', 'PROPN', 'cont'), ('URL', 'PROPN', 'URL')]
Raiz: ['regala']
(Sintagmas Mayores,Funcion sintactica): [['usuario ', 'SN: Adjunto obl'], ['estas navidades ', 'SN: Adjunto obl'], ['solidaridad ', 'SN: Objeto directo obj'], ['HASHTAG cont URL', 'SN: Sujeto nsubj']]


**Explicación tweet 3:**
- La oración "Estas navidades regala solidaridad" presenta un verbo imperativo en segunda persona del singular. Se interpreta como un anuncio en el que el sujeto de la oración es omitido.
Existe una ambigüedad en la oración "estas navidades regala solidaridad", la cual se puede generar por ejemplo si no eliminamos los ":" al preprocesar los tweets, donde el analizador interpreta a "estas navidades" como el sujeto de la oración y la solidaridad como el objeto directo. Dicho signo de puntuación preferimos eliminar debido a que tanto para este tweet como para el tweet 4 se marcaba de forma incorrecta a "usuario:" como raíz de la oración cuando debería ser "regala" como se indica en la salida.

- Retomando lo anterior, al eliminar los ":" no se soluciona el inconveniente de tomar erroneamente el sujeto de la oracion, pues este pasa a ser 'HASHTAG cont URL', cuando en realidad el mismo es omitido (sujeto tacito).

- Observamos que "cont" indica que el tweet hace referencia a otro tweet o texto citado, por lo que al reemplazar la URL por el string "URL", estamos perdiendo información en nuestro preprocesamiento que puede ser relevante para un posterior análisis.

**TWEET 4:**

In [17]:
tweet = "@DavidSummersHG: @Edurnity Oyeee...! A Madrid si quiero ir eh? Un besazo guapa! Vamossss ya contaba contigo si o si!!! Un besote enorme!!"

#Procesamos el Tweet de la parte 4, obtenemos sus sintagmas principales y mostramos el árbol generado por el analizador sintáctico:
tweet_procesado = procesar_tweet(tweet)
tweet_analizado = SintagmasMayores(tweet_procesado)
rot = raiz(tweet_procesado)
sintagmas = SintagmasMayores(tweet_procesado)
x = lema_y_pos_tag(tweet_procesado)
print("Tweet: " + str(tweet) + "\nTweet Procesado: " + str(tweet_procesado)  + "\n(palabra,POS-tag,lema): " + str(list(x))  + "\nRaiz: " + str(rot) + "\n(Sintagmas Mayores,Funcion sintactica): " + str(sintagmas) )
analisisParte4 = analisisLing(tweet_procesado)
displacy.render(analisisParte4, style="dep", jupyter=True)    


Tweet: @DavidSummersHG: @Edurnity Oyeee...! A Madrid si quiero ir eh? Un besazo guapa! Vamossss ya contaba contigo si o si!!! Un besote enorme!!
Tweet Procesado: usuario usuario oye.! a madrid si quiero ir eh? un besazo guapa! vamos ya contaba contigo si o si! un besote enorme!
(palabra,POS-tag,lema): [('usuario', 'NOUN', 'usuario'), ('usuario', 'NOUN', 'usuario'), ('oye', 'PROPN', 'oye'), ('.', 'PUNCT', '.'), ('!', 'PUNCT', '!'), ('a', 'ADP', 'a'), ('madrid', 'PROPN', 'madrid'), ('si', 'SCONJ', 'si'), ('quiero', 'VERB', 'querer'), ('ir', 'VERB', 'ir'), ('eh', 'INTJ', 'eh'), ('?', 'PUNCT', '?'), ('un', 'DET', 'uno'), ('besazo', 'NOUN', 'besazo'), ('guapa', 'ADJ', 'guapo'), ('!', 'PUNCT', '!'), ('vamos', 'INTJ', 'vamos'), ('ya', 'ADV', 'ya'), ('contaba', 'VERB', 'contar'), ('contigo', 'PRON', 'tú'), ('si', 'ADV', 'si'), ('o', 'CCONJ', 'o'), ('si', 'INTJ', 'si'), ('!', 'PUNCT', '!'), ('un', 'DET', 'uno'), ('besote', 'NOUN', 'besote'), ('enorme', 'ADJ', 'enorme'), ('!', 'PUNCT', '!')]
Rai

**Explicacion tweet 4:**
Problemas identificados:

Este tweet es dividido por el analizador sintáctico en 4 oraciones, con raíces "usuario", "quiero", "besazo", "contaba":
- En la primera oración, existe una posible ambigüedad al analizar que es la de interpretar "oye" como a un verbo, asignando a "usuario" como Sujeto. Consideramos que la POS-tag correcta para "oye" es la de Interjección, sin embargo a nosotros nos la reconoce como nombre propio, esto podría arreglarse eliminando los tres puntos suspensivos al preprocesar el texto. Si bien en dicho casos obtendriamos "oye!" en lugar de "oye.!", en general sustituir dos o mas puntos seguidos por uno solo es mas adecuado, ademas de que la expresion "...!" no es habitual ni correcta.

- En la segunda oración tenemos un verbo principal con sujeto omitido, se interpreta correctamente el sintagma "a Madrid" como adjunto y "eh?" como una interjección. Observamos que el lemma de "besazo" es incorrectamente "besazo" cuando debería ser "beso". Lo mismo ocurre con "besote", ambos aumentativos de la palabra "beso".

- El cuarto constituyente es una oración en la que se identifica una dependencia que no debería existir entre el sintagma nominal "Un besote enorme" y el verbo "contaba", interpretamos que para un buen análisis este debería ser un constituyente independiente de la oración "vamos ya contaba contigo sí o sí". 


- Al tomar nombres de usuario genérico, se puede perder información de género, semánticamente se interpreta que el segmento "Un besazo guapa!" está dirigido al usuario "@Edurnity" y no a "@DavidSummersHG" sin embargo, resulta imposible indicar a que "usuario" está dirigido después de preprocesar el texto.

- Observamos la presencia de multiples dependencias "dep" en el diagrama, lo cual refleja la poca estructura formal del tweet, lo que agrega dificultad al analizar el tweet. Esto es debido a que la dependencia "dep" indica que fué imposible determinar una relación más precisa entre constituyentes de una oración o enunciado.

# Conclusiones generales:
*   Creemos que el análisis que proporcionan las distintas herramientas utilizadas es correcto a la hora de considerar oraciones o textos con alto nivel de formalidad. Sin embargo, tal cual pudimos ver en el análisis de la parte 3, donde trabajábamos con menos formalidad, funciona de forma menos precisa al haber varias ambigüedades o casos que no fueron contemplados a la hora de desarrollar tales herramientas.
*   En cuanto a sustituciones como las de URLs y Usuarios creemos que resultan convenientes para abstraer la idea que los tweets contienen y no sobrecargarlos de información que puede no tener utilidad. 
*   Pensando en el próximo laboratorio, en cuanto a las herramientas utilizadas para este análisis consideramos que no son las más optimas para un análisis de sentimientos, sin embargo spacy nos puede volver a ser de utilidad y optamos por eliminar las _stopwords_ o "palabras vacías" de los textos, ya que estos no tienen gran aporte a la hora de analizar los sentimiento de un texto.

